In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Classification models
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Import Data into a Dataframe

In [2]:
df = pd.read_csv('heart_disease_train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'heart_disease_train.csv'

# Display The Top 5 Rows

In [ ]:
df.head()

# Display The Bottom 5 Rows

In [ ]:
df.tail()

# Display The Shape of The Data and Summary Statistics

In [ ]:
df.shape

In [ ]:
df.describe()

Here’s a **short summary** based on our dataset description:

-----------------------------------------------------------------------------------------------------------------

🔵 **General**:  
- There are **952 records** (patients).
- The target variable (`target`) is **binary** (0 or 1), suggesting a **binary classification** problem (likely predicting presence or absence of heart disease).

🔵 **Demographics**:  
- **Age**: Mean age is about **54 years** (range from ~26 to ~89 years).
- **Sex**: About **75% are male** (`sex` mean = 0.75, where 1 likely = male).

🔵 **Health Features**:
- **Chest pain type**: Mostly around types **3–4** (types are likely categorical).
- **Resting blood pressure**: Average around **132 mm Hg** but some values are suspiciously **0** (possible data error or missing value placeholder).
- **Cholesterol**: Average **209 mg/dL**, but again some entries are **0**, which is unlikely in reality (another possible missing value issue).

🔵 **Exercise and Heart Health**:
- **Max heart rate**: Average about **140 bpm**, ranging from ~58 to 202 bpm.
- **Exercise-induced angina**: Present in about **41%** of the cases (`exercise.angina` mean = 0.41).
- **Oldpeak** (ST depression): Ranges from **-2.6** to **6.2** — negative values seem unusual and may need investigation.

🔵 **Other Observations**:
- **Resting ECG** and **ST slope** seem to be **categorical features** based on limited distinct values.
- Some features (like `resting.bp` and `cholesterol`) having minimum values of **0** suggests you may need **data cleaning** (e.g., replacing 0s with NaN and imputing).

------------------------------------------------------------------------------------------------------------------

**Quick Key Issues Noted**:
- Some **zero values** where biologically impossible (blood pressure, cholesterol).
- Possibly **imbalanced** classes (target mean ≈ 0.53 → roughly balanced but check exact counts).


# Data Preprocessing

In [ ]:
df.info()

# Data Type Standardization

The dataset structure is well standardized...

# Handling Missing Data and Duplicates

The dataset did not contain any missing values , ensuring data completeness...

In [ ]:
df.isnull().sum()

# Outlier Detection and Treatment

In [ ]:
# Melt the DataFrame for easier plotting
df_melt = df.drop('ID', axis = 1)
df_melted = df_melt.melt(var_name='Variable', value_name='Value')

# Define a color palette
palette = ['#FF9999','#66B2FF','#99FF99','#FFCC99','#FF6666','#66FFFF','#99FFCC','#CCFF66','#CC99FF','#FF99CC','#C2C2F0','#FFB3E6']

# Create the box plot
plt.figure(figsize=(12, 8))
sns.boxplot(x='Variable', y='Value', data=df_melted, palette = palette)

# Add title and labels
plt.title('Before Removing Outliers')
plt.xlabel('')
plt.ylabel('Value')

# Rotate x-axis labels
plt.xticks(rotation=90)

# Show the plot
plt.show()

In [ ]:
def remove_heart_disease_outliers(df):
    columns_to_check = ['age', 'cholesterol', 'resting.bp.s', 'max_heart_rate', 'oldpeak']
    
    # Apply IQR filtering
    for column in columns_to_check:
        if column in df.columns:
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    # Apply domain-specific thresholds
    if 'age' in df.columns:
        df = df[(df['age'] >= 18) & (df['age'] <= 100)]
    
    if 'cholesterol' in df.columns:
        df = df[(df['cholesterol'] >= 100) & (df['cholesterol'] <= 400)]
    
    if 'resting.bp.s' in df.columns:
        df = df[(df['resting.bp.s'] >= 80) & (df['resting.bp.s'] <= 200)]
    
    if 'max_heart_rate' in df.columns:
        df = df[(df['max_heart_rate'] >= 60) & (df['max_heart_rate'] <= 220)]

    if 'oldpeak' in df.columns:
        df = df[(df['oldpeak'] >= 0) & (df['oldpeak'] <= 6)]
    
    return df

In [ ]:
df = df.drop('ID', axis = 1)
# Remove outliers
df_cleaned = remove_heart_disease_outliers(df)

# Melt the DataFrame for easier plotting
df_melted = df_cleaned.melt(var_name='Variable', value_name='Value')

# Define a color palette
palette = ['#FF9999','#66B2FF','#99FF99','#FFCC99','#FF6666','#66FFFF','#99FFCC','#CCFF66','#CC99FF','#FF99CC','#C2C2F0','#FFB3E6']

# Create the box plot with colored boxes
plt.figure(figsize=(12, 8))
sns.boxplot(x='Variable', y='Value', data=df_melted, palette=palette)

# Add title and labels
plt.title('After Removing Outliers')
plt.xlabel('')
plt.ylabel('Value')

# Rotate x-axis labels
plt.xticks(rotation=90)

# Show the plot
plt.show()

In [ ]:
df_cleaned.shape

After removing outliers, we are left with 767 observations

# Encoding Categorical Variables

Binary categorical features (sex, fasting blood sugar, exercise angina) were pre-encoded as 0s and
1s, simplifying their integration. Ordinal categorical features (chest pain type, resting ECG, ST slope)
were handled by the Python library's inherent capabilities, eliminating the need for manual encoding.

# Exploratory Data Analysis (EDA)

EDA is conducted to understand the distribution of features and analyze correlations between features and the target variable.

In [ ]:
# Define the features to plot
features = ['age', 'sex', 'chest.pain.type', 'resting.bp.s', 'cholesterol', 'fasting.blood.sugar',
             'resting.ecg', 'max.heart.rate', 'exercise.angina', 'oldpeak', 'ST.slope', 'target']

# Create a grid of subplots
fig, axes = plt.subplots(4, 3, figsize=(15, 15))
axes = axes.flatten()

# Plot each feature
for i, feature in enumerate(features):
    sns.histplot(df_cleaned[feature], bins=10, kde=False, ax=axes[i])
    axes[i].set_title(feature)

# Adjust layout
plt.tight_layout()
plt.show()

Based on the histograms provided, we can make several observations about the distribution of the variables:

1. **Age**:
   - The distribution appears to be approximately normal, with a peak around the middle ages (50-60 years).
   - There are fewer individuals in the younger and older age groups.

2. **Sex**:
   - This is a binary variable (0 and 1).
   - The distribution is skewed, with more individuals belonging to one category (likely male or female).

3. **Chest Pain Type**:
   - This is a categorical variable with values ranging from 1 to 4.
   - The distribution is not uniform, with some categories being more frequent than others.

4. **Resting Blood Pressure (resting_bp_s)**:
   - The distribution is approximately normal, with most values clustered around the mean.
   - There are fewer individuals with very high or very low blood pressure.

5. **Cholesterol**:
   - The distribution is right-skewed, with a long tail towards higher values.
   - Most individuals have cholesterol levels in the lower to middle range.

6. **Fasting Blood Sugar (fasting_blood_sugar)**:
   - This is a binary variable (0 and 1).
   - The distribution is highly skewed, with most individuals having a value of 0 (likely indicating normal fasting blood sugar levels).

7. **Resting ECG (resting_ecg)**:
   - This is a categorical variable with values ranging from 0 to 2.
   - The distribution is not uniform, with some categories being more frequent than others.

8. **Maximum Heart Rate (max_heart_rate)**:
   - The distribution is approximately normal, with most values clustered around the mean.
   - There are fewer individuals with very high or very low maximum heart rates.

9. **Exercise-Induced Angina (exercise_angina)**:
   - This is a binary variable (0 and 1).
   - The distribution is highly skewed, with most individuals having a value of 0 (likely indicating no exercise-induced angina).

10. **Oldpeak**:
    - The distribution is right-skewed, with most values clustered around the lower end.
    - There are fewer individuals with higher oldpeak values.

11. **ST Slope (ST_slope)**:
    - This is a categorical variable with values ranging from 0 to 2.
    - The distribution is not uniform, with some categories being more frequent than others.

12. **Target**:
    - This is a binary variable (0 and 1).
    - The distribution is skewed, with more individuals belonging to one category (likely indicating the presence or absence of a condition).

### General Observations:
- **Normality**: Some variables like age and resting blood pressure appear to be approximately normally distributed.
- **Skewness**: Variables like cholesterol, fasting blood sugar, and oldpeak are right-skewed.
- **Categorical Variables**: Variables like sex, chest pain type, resting ECG, exercise-induced angina, ST slope, and target are categorical and show non-uniform distributions.

## Heatmaps

In [ ]:
# Calculate the correlation matrix
corr_matrix = df_cleaned.corr()

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Set up the matplotlib figure
plt.figure(figsize=(12, 10))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap="coolwarm", vmin=-0.4, vmax=0.4, cbar_kws={"shrink": .8})

# Add title and labels
plt.title('Correlation Matrix of Dataset Features')
plt.xticks(rotation=90)
plt.yticks(rotation=0)

# Show the plot
plt.show()

# Model Building and Evaluation

## Logistic Regression

In [ ]:
# Define the features (X) and the target (y)
X = df_clean.drop('target', axis=1)
y = df_clean['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

### LogReg Classification Report

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

### LogReg ROC-AUC

In [ ]:
# Make predictions on the testing set
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calculate the ROC-AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Print the ROC-AUC score
print(f'ROC-AUC Score: {roc_auc}')

# Plot the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In summary, this ROC curve indicates that the model being evaluated has strong performance, as evidenced by the high ROC-AUC score and the curve's position close to the top-left corner of the plot.

## Linear Discriminant Analysis

In [ ]:
# Define the features (X) and the target (y)
X_lda = df_clean.drop('target', axis=1)
y_lda = df_clean['target']

# Split the data into training and testing sets
X_train_lda, X_test_lda, y_train_lda, y_test_lda = train_test_split(X_lda, y_lda, test_size=0.2, random_state=42)

In [ ]:
# Train the LDA model
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_lda, y_train_lda)

# Make predictions on the testing set
y_pred_lda = lda.predict(X_test_lda)

### LDA Classification Report

In [ ]:
# Evaluate the model
accuracy_lda = accuracy_score(y_test_lda, y_pred_lda)
conf_matrix_lda = confusion_matrix(y_test_lda, y_pred_lda)
class_report_lda = classification_report(y_test_lda, y_pred_lda)

# Print the results
print(f'Accuracy: {accuracy_lda}')
print('Confusion Matrix:')
print(conf_matrix_lda)
print('Classification Report:')
print(class_report_lda)

### LDA ROC-AUC

In [ ]:
# Make predictions on the testing set
y_pred_proba_lda = lda.predict_proba(X_test_lda)[:, 1]

# Calculate the ROC-AUC score
roc_auc_lda = roc_auc_score(y_test_lda, y_pred_proba_lda)

# Print the ROC-AUC score
print(f'ROC-AUC Score: {roc_auc_lda}')

# Plot the ROC curve
fpr_lda, tpr_lda, _ = roc_curve(y_test_lda, y_pred_proba_lda)
plt.figure()
plt.plot(fpr_lda, tpr_lda, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc_lda)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

## Naive Bayes (NB)

In [ ]:
# Define the features (X) and the target (y)
X_nb = df_clean.drop('target', axis=1)
y_nb = df_clean['target']

# Split the data into training and testing sets
X_train_nb, X_test_nb, y_train_nb, y_test_nb = train_test_split(X_nb, y_nb, test_size=0.2, random_state=42)

In [ ]:
# Train the Naive Bayes model
nb_model = GaussianNB()
nb_model.fit(X_train_nb, y_train_nb)

# Make predictions on the testing set
y_pred_nb = nb_model.predict(X_test_nb)

### Naive Bayes Classification Report

In [ ]:
# Evaluate the model
accuracy_nb = accuracy_score(y_test_nb, y_pred_nb)
conf_matrix_nb = confusion_matrix(y_test_nb, y_pred_nb)
class_report_nb = classification_report(y_test_nb, y_pred_nb)

# Print the results
print(f'Accuracy: {accuracy_nb}')
print('Confusion Matrix:')
print(conf_matrix_nb)
print('Classification Report:')
print(class_report_nb)

In [ ]:
# Make predictions on the testing set
y_pred_proba_nb = nb_model.predict_proba(X_test_nb)[:, 1]

# Calculate the ROC-AUC score
roc_auc_nb = roc_auc_score(y_test_nb, y_pred_proba_nb)

# Print the ROC-AUC score
print(f'ROC-AUC Score: {roc_auc_nb}')

# Plot the ROC curve
fpr_nb, tpr_nb, _ = roc_curve(y_test_nb, y_pred_proba_nb)
plt.figure()
plt.plot(fpr_nb, tpr_nb, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc_nb)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

## KNearest Neigbors

In [ ]:
# Define the features (X) and the target (y)
X_knn = df_clean.drop('target', axis=1)
y_knn = df_clean['target']

# Split the data into training and testing sets
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X_knn, y_knn, test_size=0.2, random_state=42)

In [ ]:
# Train the KNN model
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn.fit(X_train_knn, y_train_knn)

# Make predictions on the testing set
y_pred_knn = knn.predict(X_test_knn)

### KNN Classification Report

In [ ]:
# Evaluate the model
accuracy_knn = accuracy_score(y_test_knn, y_pred_knn)
conf_matrix_knn = confusion_matrix(y_test_knn, y_pred_knn)
class_report_knn = classification_report(y_test_knn, y_pred_knn)

# Print the results
print(f'Accuracy: {accuracy_knn}')
print('Confusion Matrix:')
print(conf_matrix_knn)
print('Classification Report:')
print(class_report_knn)

### KNN ROC-AUC

In [ ]:
# Make predictions on the testing set
y_pred_proba_knn = knn.predict_proba(X_test_knn)[:, 1]

# Calculate the ROC-AUC score
roc_auc_knn = roc_auc_score(y_test_knn, y_pred_proba_knn)

# Print the ROC-AUC score
print(f'ROC-AUC Score: {roc_auc_knn}')

# Plot the ROC curve
fpr_knn, tpr_knn, _ = roc_curve(y_test_knn, y_pred_proba_knn)
plt.figure()
plt.plot(fpr_knn, tpr_knn, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc_knn)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

## Decision Tree Classifier

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
# Define the features (X) and the target (y)
X_dt = df_clean.drop('target', axis=1)
y_dt = df_clean['target']

# Split the data into training and testing sets
X_train_dt, X_test_dt, y_train_dt, y_test_dt = train_test_split(X_dt, y_dt, test_size=0.2, random_state=42)

In [ ]:
# Train the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train_dt, y_train_dt)

# Make predictions on the testing set
y_pred_dt = dt_model.predict(X_test_dt)

In [ ]:
# Evaluate the model
accuracy_dt = accuracy_score(y_test_dt, y_pred_dt)
conf_matrix_dt = confusion_matrix(y_test_dt, y_pred_dt)
class_report_dt = classification_report(y_test_dt, y_pred_dt)

# Print the results
print(f'Accuracy: {accuracy_dt}')
print('Confusion Matrix:')
print(conf_matrix_dt)
print('Classification Report:')
print(class_report_dt)

In [ ]:
# Visualize the decision tree
plt.figure(figsize=(20, 10))
plot_tree(dt_model, filled=True, feature_names=X_dt.columns, class_names=['No', 'Yes'], rounded=True)
plt.show()

### Decision Tree Classifier ROC-AUC

In [ ]:
# Make predictions on the testing set
y_pred_proba_dt = dt_model.predict_proba(X_test_dt)[:, 1]

# Calculate the ROC-AUC score
roc_auc_dt = roc_auc_score(y_test_dt, y_pred_proba_dt)

# Print the ROC-AUC score
print(f'ROC-AUC Score: {roc_auc_dt}')

# Plot the ROC curve
fpr_dt, tpr_dt, _ = roc_curve(y_test_dt, y_pred_proba_dt)
plt.figure()
plt.plot(fpr_dt, tpr_dt, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc_dt)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

## Random Forest Classifier

In [ ]:
# Define the features (X) and the target (y)
X_rf = df_clean.drop('target', axis=1)
y_rf = df_clean['target']

# Split the data into training and testing sets
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.2, random_state=42)

In [ ]:
# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_rf, y_train_rf)

# Make predictions on the testing set
y_pred_rf = rf_model.predict(X_test_rf)

In [ ]:
# Evaluate the model
accuracy_rf = accuracy_score(y_test_rf, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test_rf, y_pred_rf)
class_report_rf = classification_report(y_test_rf, y_pred_rf)

# Print the results
print(f'Accuracy: {accuracy_rf}')
print('Confusion Matrix:')
print(conf_matrix_rf)
print('Classification Report:')
print(class_report_rf)

### Random Forest ROC-AUC

In [ ]:
# Make predictions on the testing set
y_pred_proba_rf = rf_model.predict_proba(X_test_rf)[:, 1]

# Calculate the ROC-AUC score
roc_auc_rf = roc_auc_score(y_test_rf, y_pred_proba_rf)

# Print the ROC-AUC score
print(f'ROC-AUC Score: {roc_auc_rf}')

# Plot the ROC curve
fpr_rf, tpr_rf, _ = roc_curve(y_test_rf, y_pred_proba_rf)
plt.figure()
plt.plot(fpr_rf, tpr_rf, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc_rf)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_rf, y_rf)

# Print the best parameters and best score
print(f'Best Parameters: {grid_search.best_params_}')
print(f'Best CV Score: {grid_search.best_score_}')

## Support Vector Classification

In [ ]:
# Define the features (X) and the target (y)
X_svc = df_clean.drop('target', axis=1)
y_svc = df_clean['target']

In [ ]:
# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_svc)

# Split the data into training and testing sets
X_train_svc, X_test_svc, y_train_svc, y_test_svc = train_test_split(X_scaled, y_svc, test_size=0.2, random_state=42)

In [ ]:
# Train the SVC model
svc_model = SVC(kernel='linear', probability=True, random_state=42)
svc_model.fit(X_train_svc, y_train_svc)

# Make predictions on the testing set
y_pred_svc = svc_model.predict(X_test_svc)

### Support Vector Classification Report

In [ ]:
# Evaluate the model
accuracy_svc = accuracy_score(y_test_svc, y_pred_svc)
conf_matrix_svc = confusion_matrix(y_test_svc, y_pred_svc)
class_report_svc = classification_report(y_test_svc, y_pred_svc)

# Print the results
print(f'Accuracy: {accuracy_svc}')
print('Confusion Matrix:')
print(conf_matrix_svc)
print('Classification Report:')
print(class_report_svc)

### Support Vector Classifier ROC-AUC

In [ ]:
# Make predictions on the testing set
y_pred_proba_svc = svc_model.predict_proba(X_test_svc)[:, 1]

# Calculate the ROC-AUC score
roc_auc_svc = roc_auc_score(y_test_svc, y_pred_proba_svc)

# Print the ROC-AUC score
print(f'ROC-AUC Score: {roc_auc_svc}')

# Plot the ROC curve
fpr_svc, tpr_svc, _ = roc_curve(y_test_svc, y_pred_proba_svc)
plt.figure()
plt.plot(fpr_svc, tpr_svc, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc_svc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Model Selection and Feature Importance

In [ ]:
# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_rf, y_train_rf)

# Get the feature importances
importances = rf_model.feature_importances_

# Create a pandas series with feature names
feature_importances = pd.Series(importances, index=X_train_rf.columns)

# Sort features by importance
feature_importances = feature_importances.sort_values(ascending=False)

## Plot Features

In [ ]:
# Plot
plt.figure(figsize=(10, 6))
feature_importances.plot(kind='bar')
plt.title('Feature Importances from Random Forest')
plt.ylabel('Importance Score')
plt.xlabel('Features')
plt.tight_layout()
plt.show()

## Saving the Model

In [ ]:
import pickle

In [ ]:
filename = 'heart_disease_model.sav'
pickle.dump(rf_model, open(filename, 'wb'))

In [ ]:
# loading the saved model
loaded_model = pickle.load(open('heart_disease_model.sav', 'rb'))

In [ ]:
for column in X_train_rf.columns:
  print(column)